## Merge by linking by rows
Comparison: 
- groupby: within one dataframe, splitting to different group by groupname
- mergeby: different dataframe, want to merge together by criteria

In [1]:
import numpy as np
from pandas import Series, DataFrame
import pandas as pd

### Merge on one key

In [2]:
# Create our dataframe 
df1 = DataFrame({'key':['X','Y','Z','X','X','Y'], 'data':[0,1,2,3,4,5]})
df1

,data,key
0,0,X
1,1,Y
2,2,Z
3,3,X
4,4,X
5,5,Y


In [3]:
df2 = DataFrame({'key':['W','Y','Z'], 'data2':[0,1,2]})
df2

,data2,key
0,0,W
1,1,Y
2,2,Z


In [4]:
# 自动选取名字相同的列来 merge， 这里是 key 这一列-- usually not use this general merge. (too many varibility)
pd.merge(df1, df2)

,data,key,data2
0,1,Y,1
1,5,Y,1
2,2,Z,2


In [7]:
# match on a specific column by passing ON para
pd.merge(df1, df2, on = 'key' ,how='inner') 
pd.merge(df1, df2, on = 'key') # default is how='inner' -->
                         # return intersection of 'key', i.e. {Y,Z} here and show all data that related to these {} for both side.

,data,key,data2
0,1,Y,1
1,5,Y,1
2,2,Z,2


In [10]:
df1

,data,key
0,0,X
1,1,Y
2,2,Z
3,3,X
4,4,X
5,5,Y


In [8]:
df2

,data2,key
0,0,W
1,1,Y
2,2,Z


In [9]:
# Merge by left--> 'key' column includes all df1's key names. 
pd.merge(df1, df2, on = 'key', how = 'left') 

,data,key,data2
0,0,X,NaN
1,1,Y,1.0
2,2,Z,2.0
3,3,X,NaN
4,4,X,NaN
5,5,Y,1.0


In [9]:
# Merge by right
pd.merge(df1, df2, on = 'key', how = 'right') 

,data,key,data2
0,1.0,Y,1
1,5.0,Y,1
2,2.0,Z,2
3,NaN,W,0


In [10]:
df1

,data,key
0,0,X
1,1,Y
2,2,Z
3,3,X
4,4,X
5,5,Y


In [11]:
df2

,data2,key
0,0,W
1,1,Y
2,2,Z


In [13]:
# Union of both keys
pd.merge(df1, df2, on = 'key', how = 'outer')

,data,key,data2
0,0.0,X,NaN
1,3.0,X,NaN
2,4.0,X,NaN
3,1.0,Y,1.0
4,5.0,Y,1.0
5,2.0,Z,2.0
6,NaN,W,0.0


- More complex case below: both df1 and df2 contains more than one X OR Y OR Z (multiple to multiple merge)

In [14]:
df2 = DataFrame({'key':['X','Y','Z','Z','X'], 'data2':[0,1,2,3,4]})
df2

,data2,key
0,0,X
1,1,Y
2,2,Z
3,3,Z
4,4,X


In [15]:
df1

,data,key
0,0,X
1,1,Y
2,2,Z
3,3,X
4,4,X
5,5,Y


In [18]:
pd.merge(df1, df2, on = 'key')   # Default how='inner'; # df 有3个X，df2 有2个X，所以出现6个X

,data,key,data2
0,0,X,0
1,0,X,4
2,3,X,0
3,3,X,4
4,4,X,0
5,4,X,4
6,1,Y,1
7,5,Y,1
8,2,Z,2
9,2,Z,3


### Merge on multiple keys

In [63]:
# Let's create two different dataframes with mutiple keys

df_left = DataFrame({'city':['LA','SF','SF','NYC'],
                     'star': ['4','4','4','5'],
                     'data1':[10,10,60,20]})

In [64]:
df_right = DataFrame({'city':['LA','SF','LA','LA'],
                     'star': ['4','4','5','3'],
                     'data2':[40,40,50,30]})

In [65]:
df_left

,city,data1,star
0,LA,10,4
1,SF,10,4
2,SF,60,4
3,NYC,20,5


In [66]:
df_right

,city,data2,star
0,LA,40,4
1,SF,40,4
2,LA,50,5
3,LA,30,3


In [67]:
pd.merge(df_left, df_right, on = ['city', 'star'], how = 'outer') # both 'city' and 'star' match, then merge.

,city,data1,star,data2
0,LA,10.0,4,40.0
1,SF,10.0,4,40.0
2,SF,60.0,4,40.0
3,NYC,20.0,5,NaN
4,LA,NaN,5,50.0
5,LA,NaN,3,30.0


In [68]:
# you can also specify what the suffix becomes
pd.merge(df_left, df_right, on = ['city'],suffixes = ('_left', '_right'))
# note: if df1 and df2's column name not match(e.g. data1 and data2), suffixes won't apply;
        # only if df1 and df2's column name match(e.g. 'star'), suffix apply--> star_left and star_right

,city,data1,star_left,data2,star_right
0,LA,10,4,40,4
1,LA,10,4,50,5
2,LA,10,4,30,3
3,SF,10,4,40,4
4,SF,60,4,40,4


## Merge on index

### Merge on single layer index

In [72]:
# Creat two dframes
df_left1 = DataFrame({'key': ['X','Y','Z','X','Y'],
                  'data_l': range(5)})
df_right1 = DataFrame({'data_r': [10, 20]}, index=['X', 'Y'])

In [73]:
df_left1

,data_l,key
0,0,X
1,1,Y
2,2,Z
3,3,X
4,4,Y


In [74]:
df_right1 # note: 'X' 'Y' here are self-defined index, just like 0,1,2,... above

,data_r
X,10
Y,20


In [75]:
#Merge on the key for the left Dframe, and the index for the right
pd.merge(df_left1,df_right1,left_on='key',right_index=True)
# note: left_on=xxx, means: use df1's xxx as joint key
    #   right_index=true, means: use df2's index(here is X and Y) as joint key
# Merge on index will carry the original index from df that does not use index as joint key (i.e.: 0,3,1,4 from df1 here)

,data_l,key,data_r
0,0,X,10
3,3,X,10
1,1,Y,20
4,4,Y,20


In [69]:
pd.merge?

In [76]:
pd.merge(df_right1,df_left1,right_on='key',left_index=True)

,data_r,data_l,key
0,10,0,X
3,10,3,X
1,20,1,Y
4,20,4,Y


### Merge on multi-layer index

In [77]:
# Hierarchal index- multi-index--> see df_right
df_left_hr = DataFrame({'key1': ['SF','SF','SF','LA','LA'],
                   'key2': [10, 20, 30, 20, 30],
                   'data_set': np.arange(5.)})
df_right_hr = DataFrame(np.arange(10).reshape((5, 2)),
                   index=[['LA','LA','SF','SF','SF'],
                          [20, 10, 10, 10, 20]],
                   columns=['col_1', 'col_2'])

In [78]:
df_left_hr

,data_set,key1,key2
0,0.0,SF,10
1,1.0,SF,20
2,2.0,SF,30
3,3.0,LA,20
4,4.0,LA,30


In [79]:
df_right_hr

col_1  col_2
LA 20      0      1
   10      2      3
SF 10      4      5
   10      6      7
   20      8      9

In [81]:
# Merge the left by using keys and the right by its index
pd.merge(df_left_hr,df_right_hr,left_on=['key1','key2'],right_index=True) 
# same as merge on multi-keys, only if both key1, key2 pair matches index layer 1 and layer 2, it merges. 

,data_set,key1,key2,col_1,col_2
0,0.0,SF,10,4,5
0,0.0,SF,10,6,7
1,1.0,SF,20,8,9
3,3.0,LA,20,0,1


In [82]:
# note: order of keys should also match index order, from outer layer to inner layer, otherwise, if I switch key1 and key2,
#see result below-> can't merge
pd.merge(df_left_hr,df_right_hr,left_on=['key2','key1'],right_index=True) 

,data_set,key1,key2,col_1,col_2


In [83]:
# Union by choosing 'outer' method
pd.merge(df_left_hr,df_right_hr,left_on=['key1','key2'],right_index=True,how='outer')

,data_set,key1,key2,col_1,col_2
0,0.0,SF,10,4.0,5.0
0,0.0,SF,10,6.0,7.0
1,1.0,SF,20,8.0,9.0
2,2.0,SF,30,NaN,NaN
3,3.0,LA,20,0.0,1.0
4,4.0,LA,30,NaN,NaN
4,NaN,LA,10,2.0,3.0
